# Ejercicios 

Los datos adjuntos a esta lección forman parte de la base de datos [NeurIPS 2020] Data Science for COVID-19 (DS4C) disponible en Kaggle.\
Estos datos hacen referencia a los casos de contagio de covid-19 en Corea del Sur. \
El archivo csv Case contiene los casos reportados y el archivo csv PatientInfo contiene la información de los pacientes.

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import desc
from pyspark.sql.functions import count
from pyspark.storagelevel import StorageLevel

# directorio donde estan los datos
path = 'files/data/'

In [3]:
spark = SparkSession.builder.getOrCreate()
casos = spark.read.option('header','true').option('inferSchema','true').csv(path+'Case.csv')
pacientes_info = spark.read.option('header','true').option('inferSchema','true').csv(path+'PatientInfo.csv')

In [7]:
casos.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- group: boolean (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- confirmed: integer (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)



In [6]:
casos = casos.withColumnRenamed(' case_id','case_id')

## 1. A partir del archivo csv Case, determine las tres ciudades con más casos confirmados de la enfermedad. La salida debe contener tres columnas: provincia, ciudad y casos confirmados. El resultado debe contener exactamente los tres nombre de ciudades con más casos confirmados ya que no se admiten otros valores.

In [8]:
casos.show(5)

+-------+--------+------------+-----+--------------------+---------+---------+----------+
|case_id|province|        city|group|      infection_case|confirmed| latitude| longitude|
+-------+--------+------------+-----+--------------------+---------+---------+----------+
|1000001|   Seoul|  Yongsan-gu| true|       Itaewon Clubs|      139|37.538621|126.992652|
|1000002|   Seoul|   Gwanak-gu| true|             Richway|      119| 37.48208|126.901384|
|1000003|   Seoul|     Guro-gu| true| Guro-gu Call Center|       95|37.508163|126.884387|
|1000004|   Seoul|Yangcheon-gu| true|Yangcheon Table T...|       43|37.546061|126.874209|
|1000005|   Seoul|   Dobong-gu| true|     Day Care Center|       43|37.679422|127.044374|
+-------+--------+------------+-----+--------------------+---------+---------+----------+
only showing top 5 rows



In [16]:
# Top 3
casos.filter((col('city') != '-') & (col('city') != 'from other city'))\
    .orderBy(desc('confirmed'))\
    .select('province','city','confirmed')\
    .limit(3)\
    .show()

+--------+------------+---------+
|province|        city|confirmed|
+--------+------------+---------+
|   Daegu|      Nam-gu|     4511|
|   Daegu|Dalseong-gun|      196|
|   Seoul|  Yongsan-gu|      139|
+--------+------------+---------+



## 2. Cree un dataframe a partir del archivo csv PatientInfo. Asegúrese de que su dataframe no contenga pacientes duplicados.
### a. ¿Cuántos pacientes tienen informado por quién se contagiaron(columna infected_by)? Obtenga solo los pacientes que tengan informado por quién se contagiaron.

In [17]:
pacientes_info.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: date (nullable = true)
 |-- released_date: date (nullable = true)
 |-- deceased_date: date (nullable = true)
 |-- state: string (nullable = true)



In [18]:
pacientes_info.show(10)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       NULL|            75|        2020-01-22|    2020-01-23|   2020-02-05|         NULL|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       NULL|            31|              NULL|    2020-01-30|   2020-03-02|         NULL|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              NULL|    2020-01-30|   202

In [19]:
# asegurar que no hay pacientes duplicados
pacientes_info = pacientes_info.dropDuplicates(['patient_id'])

In [22]:
# Pacientes con informacion por quien fue contagiado
pacientes_info.select(count('infected_by').alias('conteo')).show()

+------+
|conteo|
+------+
|  1346|
+------+



In [23]:
# pacientes con la informacion de  por quien fue contagiado
pacientes_info_contagios = pacientes_info.na.drop(subset=['infected_by'])
pacientes_info_contagios.select(count('infected_by').alias('conteo')).show()

+------+
|conteo|
+------+
|  1346|
+------+



### b. A partir de la salida del inciso anterior obtenga solo los pacientes femeninos. La salida no debe contener las columnas released_date y deceased_date.

In [27]:
final_df = pacientes_info_contagios.filter(col('sex') == 'female').drop('released_date','deceased_date')
final_df.show()

+----------+------+---+-------+--------+-------------+--------------------+-----------+--------------+------------------+--------------+--------+
|patient_id|   sex|age|country|province|         city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|   state|
+----------+------+---+-------+--------+-------------+--------------------+-----------+--------------+------------------+--------------+--------+
|1000000005|female|20s|  Korea|   Seoul|  Seongbuk-gu|contact with patient| 1000000002|             2|              NULL|    2020-01-31|released|
|1000000006|female|50s|  Korea|   Seoul|    Jongno-gu|contact with patient| 1000000003|            43|              NULL|    2020-01-31|released|
|1000000010|female|60s|  Korea|   Seoul|  Seongbuk-gu|contact with patient| 1000000003|             6|              NULL|    2020-02-05|released|
|1000000014|female|60s|  Korea|   Seoul|    Jongno-gu|contact with patient| 1000000013|            27|        2020-02-06|   

### c. Establezca el número de particiones del dataframe resultante del inciso anterior en dos. Escriba el dataframe resultante en un archivo parquet. La salida debe estar particionada por la provincia y el modo de escritura debe ser overwrite.

In [ ]:
#final_df.coalesce(2).write.partitionBy('province').mode('overwrite').parquet('./dat/salida')